# SmolVLA Fine-tuning on Google Colab

This notebook fine-tunes the SmolVLA model (`lerobot/smolvla_base`) on your dataset using lerobot's training framework.

**Model**: [lerobot/smolvla_base](https://huggingface.co/lerobot/smolvla_base) (450M parameters)

**Dataset**: HenryZhang/Group11_data_1763075740.884942

**Reference**: [SmolVLA Blog Post](https://huggingface.co/blog/smolvla)

**Author**: JESSI11111


## Step 1: Setup Environment

First, make sure you're using a GPU runtime:
- Go to `Runtime` → `Change runtime type` → Select `GPU` (preferably A100 for Colab Pro)
- Then run the cells below to install dependencies


In [1]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("⚠️ Warning: No GPU detected. Training will be very slow!")

CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
CUDA version: 12.6


Install the required dependencies:

In [2]:
# Clone LeRobot repository
!git clone https://github.com/huggingface/lerobot.git
%cd lerobot

# Install LeRobot with SmolVLA dependencies
%pip install -e ".[smolvla]"

# Verify installation
!python -c "import lerobot; print('LeRobot installed successfully!')"

Cloning into 'lerobot'...
remote: Enumerating objects: 43233, done.
remote: Counting objects: 100% (708/708), done.
remote: Compressing objects: 100% (282/282), done.
remote: Total 43233 (delta 606), reused 444 (delta 420), pack-reused 42525 (from 4)
Receiving objects: 100% (43233/43233), 220.55 MiB | 47.73 MiB/s, done.
Resolving deltas: 100% (27820/27820), done.
Filtering content: 100% (45/45), 69.03 MiB | 51.33 MiB/s, done.
/content/lerobot
Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 

LeRobot installed successfully!


## Step 2: Configuration

Configure your training parameters. Adjust these as needed:


In [13]:
# Training configuration
# Adjust these parameters based on your needs and GPU memory

CONFIG = {
    "policy_path": "lerobot/smolvla_base",  # Pretrained model from HuggingFace
    "dataset_repo_id": "HenryZhang/Group11_data_1763075740.884942",  # Your SO101 dataset
    "batch_size": 4,  # Increased to 4 as requested (A100 can handle this)
    "steps": 20000,  # Training steps
}

print("Training Configuration:")
print("=" * 50)
for key, value in CONFIG.items():
    print(f"  {key}: {value}")
print("=" * 50)
print("\n💡 Tip: If you encounter out-of-memory (OOM) errors, reduce batch_size to 16 or 8")

Training Configuration:
  policy_path: lerobot/smolvla_base
  dataset_repo_id: HenryZhang/Group11_data_1763075740.884942
  batch_size: 4
  steps: 20000

💡 Tip: If you encounter out-of-memory (OOM) errors, reduce batch_size to 16 or 8


python lerobot/scripts/train.py \
  --policy.path=lerobot/smolvla_base \
  --dataset.repo_id=HenryZhang/Group11_data_1763075740.884942 \
  --batch_size=1 \
  --steps=20000  # 10% of training budget

## Step 3: Start Training

Now we'll start the fine-tuning process using lerobot's training script. This may take a while depending on your GPU and number of steps.


In [14]:
# Fine-tune the pretrained model
# Note: Adjust batch_size based on your GPU memory (A100 can handle 32-64)
# If you get OOM errors, reduce batch_size to 16 or 8

# Ensure we're in the lerobot directory
import os
os.chdir('/content/lerobot')

# The training script is at src/lerobot/scripts/lerobot_train.py
# Run it directly using the file path
print("Starting training...")
print(f"Current directory: {os.getcwd()}")

# Run the training script directly
!python src/lerobot/scripts/lerobot_train.py \
  --policy.type=smolvla \
  --policy.pretrained_path={CONFIG['policy_path']} \
  --policy.repo_id=smolvla_finetuned \
  --dataset.repo_id={CONFIG['dataset_repo_id']} \
  --batch_size={CONFIG['batch_size']} \
  --steps={CONFIG['steps']} \
  --optimizer.lr=5e-5 \
  --save_freq=5000 \
  --eval_freq=5000  # Evaluate every 5000 steps to monitor progress

Starting training...
Current directory: /content/lerobot
2025-11-26 03:49:19.130633: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-26 03:49:19.148466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764128959.169876   16185 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764128959.176444   16185 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764128959.193078   16185 computation_placer.cc:177] computation placer al

It took 70 minutes for training execution

LeRobot (and most modern Hugging Face libraries) uses .safetensors. This is the modern equivalent of .pt. It stores the exact same weights but is safer and loads faster. You don't need to convert it; it works natively with LeRobot and Transformers.

In [16]:
#push finetuned model to huggingface
import os
from huggingface_hub import HfApi
from pathlib import Path

# 1. Find the latest training output directory
output_root = Path("outputs/train")
if not output_root.exists():
    print("No output directory found. Did the training run?")
else:
    # Find all subdirectories (recursively or structured)
    # Structure seems to be outputs/train/date/time_name
    # We'll just find the most recently modified folder that contains 'checkpoints' or config
    all_dirs = [d for d in output_root.rglob("*") if d.is_dir() and "smolvla" in d.name]

    if not all_dirs:
        print("No 'smolvla' training output found.")
    else:
        # Sort by modification time, newest first
        latest_run_dir = max(all_dirs, key=os.path.getmtime)
        print(f"Found latest training run: {latest_run_dir}")

        # 2. Define your repository ID
        api = HfApi()
        try:
            user_info = api.whoami()
            username = user_info['name']
            repo_name = "smolvla_finetuned"
            repo_id = f"{username}/{repo_name}"

            print(f"Uploading to: https://huggingface.co/{repo_id}")

            # 3. Create Repo and Upload
            api.create_repo(repo_id=repo_id, exist_ok=True, repo_type="model")

            api.upload_folder(
                folder_path=str(latest_run_dir),
                repo_id=repo_id,
                repo_type="model",
                ignore_patterns=["wandb/*"]  # Ignore wandb logs if any
            )
            print("✅ Upload complete! check your repo link above.")

        except Exception as e:
            print(f"❌ Upload failed: {e}")
            print("Make sure you ran the login cell above!")

Found latest training run: outputs/train/2025-11-26/03-49-28_smolvla


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading to: https://huggingface.co/JESSI11111/smolvla_finetuned


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

...

  ...timizer_state.safetensors:   0%|          |  607kB /  413MB            

  ...timizer_state.safetensors:   0%|          |  607kB /  413MB            

  ...zer_processor.safetensors:  15%|#4        |   484B / 3.26kB            

  ...zer_processor.safetensors:  15%|#4        |   484B / 3.26kB            

✅ Upload complete! check your repo link above.


In [ ]:
# Other team can load the model: They just need lerobot installed.
# Example for your team to load the model directly from Hugging Face
from lerobot.common.policies.act.modeling_act import ACTPolicy

# They can load it using your Repo ID
repo_id = "JESSI11111/smolvla_finetuned"

print(f"Loading model from {repo_id}...")
policy = ACTPolicy.from_pretrained(repo_id)

print("✅ Model loaded successfully! Ready for inference.")

In [18]:
#When you need to retrain, you can simply run pip install -r requirements.txt to restore this setup.

from google.colab import drive
import shutil
import os

# 1. Generate the requirements file from current environment
print("Generating requirements.txt...")
!pip freeze > requirements.txt

# 2. Mount Drive (checks if already mounted)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 3. Define destination (using the main backup folder)
backup_dir = '/content/drive/MyDrive/lerobot_backup'
os.makedirs(backup_dir, exist_ok=True)

# 4. Copy to Drive
dest_path = os.path.join(backup_dir, 'requirements.txt')
shutil.copy('requirements.txt', dest_path)

print(f"✅ Dependencies saved to: {dest_path}")
print(f"To restore later, run: !pip install -r {dest_path}")

Generating requirements.txt...
Mounted at /content/drive
✅ Dependencies saved to: /content/drive/MyDrive/lerobot_backup/requirements.txt
To restore later, run: !pip install -r /content/drive/MyDrive/lerobot_backup/requirements.txt


In [ ]:
from google.colab import drive
import shutil
import os

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define source (your training output) and destination (your Drive)
source_dir = '/content/lerobot/outputs/train'
dest_dir = '/content/drive/MyDrive/lerobot_backup/train'

# 3. Copy files
if os.path.exists(source_dir):
    print(f"Backing up {source_dir} to {dest_dir}...")
    # Copy the directory tree
    shutil.copytree(source_dir, dest_dir, dirs_exist_ok=True)
    print("✅ Backup complete! Your files are safe in Google Drive.")
else:
    print("❌ Source directory not found. Did you run the training?")